<a href="https://colab.research.google.com/github/cmari038/Language-Translator/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm
#!pip install collections

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torch.utils.data import random_split
from torchtext.vocab import vocab
import io
import torchtext
from collections import Counter, OrderedDict

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

dataset = 'https://raw.githubusercontent.com/cmari038/Language-Translator/main/data.csv'
data = pd.read_csv(dataset)

# processing data

english_tokenizer = get_tokenizer('spacy', language = 'en')
spanish_tokenizer = get_tokenizer('spacy', language= 'es')

#print(data)

train = data.sample(frac=0.7, random_state=50)
validate = data.drop(train.index).sample(frac=0.1, random_state = 25)
test = data.drop(validate.index)

#train, validate, test = random_split(data, [0.7*len(data), 0.10*len(data), 0.20*len(data)])

counter1 = Counter()
counter2 = Counter()

for sentence in data['english']:
  counter1.update(english_tokenizer(sentence))

for sentence in data['spanish']:
  counter2.update(spanish_tokenizer(sentence))

en_dict = OrderedDict(counter1.most_common())
es_dict = OrderedDict(counter2.most_common())

vocab1 = vocab(en_dict, specials = ['<unk>', '<pad>', '<bos>', '<eos>'])
vocab2 = vocab(es_dict, specials = ['<unk>', '<pad>', '<bos>', '<eos>'])

#vocab1.set_default_index(0)
#vocab2.set_default_index(1)

# make tensors

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "es" could not be loaded, trying "es_core_news_sm" instead
  warnings.warn(


OrderedDict([('.', 102819), ('de', 22536), ('que', 21277), ('Tom', 19401), ('a', 18820), ('?', 16018), ('¿', 16010), ('la', 15208), ('en', 12226), ('el', 11710), ('no', 10974), ('es', 10428), ('un', 9731), ('No', 9213), (',', 7401), ('se', 6606), ('una', 6123), ('lo', 5618), ('Él', 5025), ('por', 4965), ('me', 4707), ('con', 4703), ('está', 4430), ('su', 4361), ('Mary', 3990), ('El', 3918), ('para', 3805), ('los', 3738), ('y', 3628), ('le', 3615), ('Me', 3513), ('Ella', 3513), ('más', 3436), ('mi', 3419), ('al', 3125), ('las', 3098), ('La', 2945), ('qué', 2700), ('te', 2671), ('muy', 2360), ('tu', 2244), ('del', 2147), ('él', 2131), ('Por', 2041), ('tiene', 2036), ('Es', 1992), ('Qué', 1987), ('casa', 1943), ('hacer', 1827), ('estaba', 1812), ('aquí', 1794), ('Yo', 1790), ('A', 1746), ('como', 1739), ('fue', 1716), ('eso', 1588), ('algo', 1573), ('tiempo', 1559), ('Estoy', 1504), ('gusta', 1504), ('esta', 1480), ('Te', 1463), ('Mi', 1459), ('favor', 1418), ('ir', 1408), ('ha', 1391), (

In [ ]:
class RNN_Encoder(nn.Module):
    def __init__(self, input, hidden_dimension, dropout_p=0.1):
        #super(RNN_Encoder, self).__init()__()
        self.hidden_dimension = hidden_dimension
        #self.num_layers = num_layers
        #self.rnn = nn.RNN(input, hidden_dimension, num_layers, batch_first=True, nonlinearity='relu')
        #self.fc = nn.Linear(hidden_dimension, output)
        self.embed = nn.Embedding(input, hidden_dimension)
        self.gru = nn.GRU(hidden_dimension, hidden_dimension, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        #h = torch.zeros(self.num_layers, x.size(0), self.hidden_dimension)
        #out, hn = self.rnn(x, h)
        #out = self.fc(out[:, -1, :])
        embedded = self.dropout(self.embedding(input))
        out, hidden = self.gru(embedded)
        return out

class RNN_Deocder(nn.module):
    def __init__(self, hidden_dimension, output):
      #super(RNN_Decoder, self).__init()__()
      self.embed = nn.Embedding(output, hidden_dimension)
      self.gru = nn.GRU(hidden_dimension, hidden_dimension, batch_first=True)
      self.out = nn.Linear(hidden_dimension, output)


Cuda
